# Shelter Animal Outcomes

## Usando más algoritmos

Retomaremos el método usado en el ejemplo del Titanic, pero ahora para otro caso: el destino de los animales en un albergue. A diferencia del caso del Titanic donde solo había 2 posibles finales para cada pasajero (sobrevivió o no sobrevivió), cada animal puede resultar adoptado, transferido, regresado a su dueño, sacrificado o muerto.

Al ver que varios códigos de otros usuarios se concentraban más en analizar los datos que pasarlos por un algoritmo, decidí comenzar usando el mismo método que en el Titanic, pero al final probé con otros algoritmos para hacer comparaciones.

El Leaderboard de este ejemplo en Kaggle
https://www.kaggle.com/c/shelter-animal-outcomes/leaderboard

El usuario con el que probé mis resultados es *a-s-ulloa92*

Comenzamos importando las librerias necesarias

In [2]:
import numpy as np
import pandas as pd
import re
from sklearn import cross_validation as cv
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier

from sklearn.neighbors import KNeighborsClassifier                         # KNN
from sklearn.svm import SVC                                                # SVM
from sklearn.ensemble import RandomForestClassifier                        # Bósque aleatorios
from sklearn.ensemble import AdaBoostClassifier                            # ADA Boost
from sklearn.naive_bayes import GaussianNB                                 # Naive bayes
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis       # Logística sin regularización
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis    # Logística con polinomio de orden 2
from sklearn.linear_model import  LogisticRegression                       # Logística con regularización

Importamos los datos de entrenamiento y prueba.

In [3]:
print('Consiguiendo los datos de entrenamiento y el conjunto de prueba...')
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

print 'Datos de entrenamiento:', len(train)

print 'Datos de prueba:', len(test)

Consiguiendo los datos de entrenamiento y el conjunto de prueba...
Datos de entrenamiento: 26729
Datos de prueba: 11456


Usaremos algunas funciones sencillas para pasar a enteros o flotantes los atributos de los animales.

In [4]:
def texto_a_edad(age):
    """
    Consigue la edad de un string y regresa un float. El tiempo es en meses y se considera que un mes son 4 semanas
    """
    if pd.isnull(age):
        return age
    
    if 'year' in age:
        return 12*int(re.search(r'\d+', age).group())
    
    if 'month' in age:
        return int(re.search(r'\d+', age).group())
    
    if 'week' in age:
        return 0.25*int(re.search(r'\d+', age).group())
    
    return 0

def esCruza(breed):
    """
    Regresa 1 si el perro es una cruza (no de raza pura); 0 en otro caso
    """

    if 'Mix' in breed:
        return 1
    
    if '/' in breed:
        return 1
    else:
        return 0
    
def perro_o_gato(x):
    """
    Si el animal es un perro, regresa 1; si es un gato, regresa 0
    """
    if 'Dog' in x: return 1
    else: return 0

def get_sex(x):
    """
    Si el animal es macho, regresa 2; si es hembra, regresa 1; si no se conoce su sexo, regresa 0
    """
    if x.find('Male') >= 0: return 2
    if x.find('Female') >= 0: return 1
    return 0

def get_neutered(x):
    """
    Si el animal está esterilizado, regresa 2; si no lo está, regresa 1; si se desconoce, se regresa 0
    """
    x = str(x)
    if x.find('Spayed') >= 0: return 2
    if x.find('Neutered') >= 0: return 2
    if x.find('Intact') >= 0: return 1
    return 0

Con ayuda de estas funciones, declararemos otra para usarlas y además eliminar campos vacíos.

In [5]:
def fix_data(animals):
    # Llena campos vacíos
    animals["Name"] = animals["Name"].fillna(0)
    animals["SexuponOutcome"] = animals["SexuponOutcome"].fillna('Unknown')
    
    
    #Pasa a enteros la edad
    animals['Age'] = animals.AgeuponOutcome.apply(texto_a_edad)
    
    #Determina si el animal es cruza o no
    animals['Breed'] = animals.Breed.apply(esCruza)
    
    #Asigna un entero dependiendo del sexo del animal
    animals['Sex'] = animals.SexuponOutcome.apply(get_sex)
    
    #Asigna un entero dependiendo de si el animal está o no esterilizado
    animals['Neutered'] = animals.SexuponOutcome.apply(get_neutered)
    
    #Asigna un valor dependiendo de si el animal es perro o gato
    animals['Type'] = animals.AnimalType.apply(perro_o_gato)
    
    #Elimina campos en blanco de la columna Age
    animals["Age"] = animals["Age"].fillna(animals["Age"].mean())
    
    
    
    # Asigna valores numéricos a los datos para facilitar cálculos
    animals.loc[animals["Name"] != 0, "Name"] = 1
    
    return animals

Ahora sigue usar esta función para "arreglar" tanto los datos de entrenamiento como los de prueba.

In [6]:
train_data = fix_data(train)
test_data  = fix_data(test)

Declaramos la función que usaremos más adelante para guardar nuestrar predicciones. Comenzamos con la columna de ID y las siguientes corresponden a cada posible estado en el que puede terminar cada animal. En cada fila estas columnas tendrán el valor de 0 con excepción de una, dependiendo del estado final correspondiente de cada uno.

In [7]:
def create_submission(dtc, train, test, predictors, filename):
    dtc.fit(train[predictors], train["OutcomeType"])
    predictions = dtc.predict(test[predictors])
    
    temp = pd.DataFrame({
        "ID": test["ID"],
        "Type": predictions,
        "Return_to_owner":0,
        "Euthanasia":0,
        "Transfer":0,
        "Died":0,
        "Adoption":0
    })
    
    temp.loc[temp["Type"] == "Return_to_owner", "Return_to_owner"] = 1
    temp.loc[temp["Type"] == "Euthanasia", "Euthanasia"] = 1
    temp.loc[temp["Type"] == "Transfer", "Transfer"] = 1
    temp.loc[temp["Type"] == "Died", "Died"] = 1
    temp.loc[temp["Type"] == "Adoption", "Adoption"] = 1

    temp = temp.drop('Type', 1)

    temp.to_csv(filename, index=False)

Guardaremos en una lista los datos que usaremos para nuestras predicciones. Estos campos son:
* Nombre
* Raza
* Sexo
* Edad
* Esterilizado
* Tipo (Si es perro o gato)

In [8]:
predictors = ["Name", "Breed", "Sex", "Age", "Neutered", "Type"]

En *sklearn* los árboles incluyen la función *score*, la cual regresa la precisión de los datos de prueba, dando como parámetros estos datos y las clases a las que pertenece cada uno. Usaremos esto para calcular la profundidad (llamada *max_depth* por el algoritmo) que nos dé los resultados más acertados, probando cada caso del 1 al 100.

In [11]:
max_score = 0
best_n = 0
for n in range(1,100):
    dtc_scr = 0.
    dtc = DecisionTreeClassifier(max_depth=n)
    for train, test in KFold(len(train_data), n_folds=10, shuffle=True):
        dtc.fit(train_data[predictors].T[train].T, train_data["OutcomeType"].T[train].T)
        dtc_scr += dtc.score(train_data[predictors].T[test].T, train_data["OutcomeType"].T[test].T)/10
    if dtc_scr > max_score:
        max_score = dtc_scr
        best_n = n
print(best_n, max_score)

(6, 0.64690055915105815)


Otro parámetro del arbol de decisión es *min_samples_split*, el cual es el mínimo número de muestras necesarias para partir un nodo. Al igual que con la profundidad, buscaremos el mejor valor para nuestro algoritmo.

In [13]:
max_score = 0
best_s = 0
for s in range(1,100):
    dtc_scr = 0.
    dtc = DecisionTreeClassifier(min_samples_split=s)
    for train, test in KFold(len(train_data), n_folds=10, shuffle=True):
        dtc.fit(train_data[predictors].T[train].T, train_data["OutcomeType"].T[train].T)
        dtc_scr += dtc.score(train_data[predictors].T[test].T, train_data["OutcomeType"].T[test].T)/10
    if dtc_scr > max_score:
        max_score = dtc_scr
        best_s = s
print(best_s, max_score)

(76, 0.64581578985687449)


Luego de esto ya podemos crear nuestro arbol de decisión. El código original usaba como criterio *entropy* y un separador de nodos *random*. El criterio puede cambiarse a *gini* para usar la impureza de Gini en vez de ganancia de información, y como estrategia para el separador se puede cambiar a *best* para elegir la mejor separación (*random* usa la mejor separación aleatoria).

In [14]:
print('Haciendo predicciones...')
dtc = DecisionTreeClassifier(max_depth=best_n, min_samples_split=best_s, criterion='entropy', splitter='random')
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "sheltersurvivors.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


¿Qué pasa si usamos otro algoritmo? Ahora probaremos con un Naive-Bayes gaussiano

In [15]:
print('Haciendo predicciones...')
dtc = GaussianNB()
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "sheltersurvivors-NB.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


El resultado fue un poco peor que el primero, pero ayuda a darnos cuenta que el algoritmo que usemos es importante y afectará nuestros resultados.

Haremos unas últimas pruebas con regresión logística. Primero, con los valores por default del algoritmo.

In [16]:
print('Haciendo predicciones...')
dtc = LogisticRegression()
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "sheltersurvivors-log1.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


Y ahora con una regularización un poco más estricta.

In [17]:
print('Haciendo predicciones...')
dtc = LogisticRegression(C = 0.5)
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "sheltersurvivors-log2.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


Un último intento, pero con menos regularización.

In [18]:
print('Haciendo predicciones...')
dtc = LogisticRegression(C = 2)
print('Creando archivo Excel...')
create_submission(dtc, train_data, test_data, predictors, "sheltersurvivors-log3.csv")
print('Listo.')

Haciendo predicciones...
Creando archivo Excel...
Listo.


Ordenando los algoritmos por el resultado obtenido, de mejor a peor, tendríamos:
* Árboles de decisión
* Regresión logística (regularización alta)
* Regresión logística (regularización normal o baja)
* Naive-Bayes Gaussiano